In [ ]:
pip install keras streamlit pillow numpy h5py

In [1]:
!pip -q install streamlit
!pip -q install pyngrok

In [13]:
#Keras pretrained model
!wget https://github.com/Coachnmomof3/UCB_COVID_Prediction_Model/blob/Bijan_Samimi_ml/Models/VGG16_final_model/VGG16_model.h5

--2021-01-14 00:09:26--  https://github.com/Coachnmomof3/UCB_COVID_Prediction_Model/blob/Bijan_Samimi_ml/Models/VGG16_final_model/VGG16_model.h5
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘VGG16_model.h5’

VGG16_model.h5          [ <=>                ]  92.04K  --.-KB/s    in 0.03s   

2021-01-14 00:09:27 (3.21 MB/s) - ‘VGG16_model.h5’ saved [94246]



In [17]:
%%writefile app.py
import streamlit as st
import cv2
from PIL import Image, ImageOps
import numpy as np 

import tensorflow as tf

model_name = 'VGG16_model.h5'
model = tf.keras.models.load_model(model_name)

st.write("""
         # COVID-19 and Viral Pneumonia Predictions
         """
         )
st.write("This is a simple image classification web app to predict if a patient has COVID-19, Viral Pneumonia or is Healthy from chest X-Rays")
file = st.file_uploader("Please upload an image file", type=["jpg", "png"])

def import_and_predict(image_data, model):
    size = (224,224)    
    image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
    image = np.asarray(image)
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_resize = (cv2.resize(img, dsize=(224, 224),    interpolation=cv2.INTER_CUBIC))/255.
        
    img_reshape = img_resize[np.newaxis,...]
    
    prediction = model.predict(img_reshape)
        
    return prediction

if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    prediction = import_and_predict(image, model)
    
    max = np.max(prediction)
    
    if prediction[0][0]==max:
      st.text("The prediction is COVID")
    if prediction[0][1]==max:
      st.text("The prediction is NORMAL")
    if prediction[0][2]==max:
      st.text("The prediction is Viral Pneumonia")
    
    st.text("Likelihood 0: COVID, 1: NORMAL, 2: Viral Pneumonia")
    st.write(prediction)

Overwriting app.py


In [16]:
from pyngrok import ngrok
public_url = ngrok.connect(port='80')
print('Link to web app:')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null

Link to web app:
NgrokTunnel: "http://5832acf4a0a6.ngrok.io" -> "http://localhost:80"
2021-01-14 00:39:58.638115: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-14 00:40:00.281792: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-14 00:40:00.282901: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-14 00:40:00.294531: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-01-14 00:40:00.294600: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (1e59674acbe5): /proc/driver/nvidia/version does not exist
2021-01-14 00:40:00.295439: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_device